In [1]:
from fls import *
import sys
import os
import re
from collections import namedtuple
__VERSION__ = "1.1"
__DATE__ = "11/Dec/2022"

# Convert NBTest

Converts files `NBTest_9999_Comment.py -> test_9999_Comment.py` suitable for `pytest`

In [2]:
print(f"NBTestConvert v{__VERSION__} {__DATE__}")

NBTestConvert v1.1 11/Dec/2022


## Get script path and set paths

In [3]:
sys.argv[0].rsplit("/", maxsplit=1)

['/Users/skl/opt/anaconda3/lib/python3.8/site-packages',
 'ipykernel_launcher.py']

In [4]:
sys.argv[0].rsplit("/", maxsplit=1)[-1]

'ipykernel_launcher.py'

In [5]:
if sys.argv[0].rsplit("/", maxsplit=1)[-1]=="ipykernel_launcher.py":
    JUPYTER = True
    SCRIPTPATH = os.getcwd()
else:
    JUPYTER = False
    SCRIPTPATH = os.path.dirname(os.path.realpath(sys.argv[0]))

In [6]:
SRCPATH = os.path.join(SCRIPTPATH, "")
TRGPATH = os.path.join(SCRIPTPATH, "../../Carbon/tests/nbtest")

In [7]:
print("JUPYTER", JUPYTER)
print("SCRIPTPATH", SCRIPTPATH)
print("SRCPATH", SRCPATH)
print("TRGPATH", TRGPATH)
print("---")

JUPYTER True
SCRIPTPATH /Users/skl/REPOES/Bancor/CarbonSimulator/resources/NBTest
SRCPATH /Users/skl/REPOES/Bancor/CarbonSimulator/resources/NBTest/
TRGPATH /Users/skl/REPOES/Bancor/CarbonSimulator/resources/NBTest/../../Carbon/tests/nbtest
---


## Generate the list of files

In [8]:
rawlist = os.listdir(SRCPATH)
rawlist.sort()
#rawlist

In [9]:
dr_nt = namedtuple("datarecord_nt", "tid, comment, fn, outfn")
def filterfn(fn):
    """
    takes fn and returns either filelist_nt or None 
    """
    nxsplit = fn.rsplit(".", maxsplit=1)
    if len(nxsplit) < 2: return None
    if not(nxsplit[1].lower()=="py"): return None
    fnsplit = nxsplit[0].split("_")
    if not len(fnsplit) in [2,3]: return None
    if not fnsplit[0] == "NBTest": return None
    tid = fnsplit[1]
    try:
        comment = fnsplit[2]
    except IndexError:
        comment = ""
    outfn = f"test_{tid}_{comment}.py"
    return dr_nt(tid=tid, comment=comment, fn=fn, outfn=outfn)

assert filterfn("README") is None
assert filterfn("NBTest_0000_Bla.ipynb") is None
assert filterfn("NBTest_0000.py")
assert filterfn("Test_0000_Bla.py") is None
assert filterfn("NBTest_1.10.4_Bla.py").tid == "1.10.4"
assert filterfn("NBTest_1.py").comment == ""
filterfn("NBTest_0000_Bla.py")

datarecord_nt(tid='0000', comment='Bla', fn='NBTest_0000_Bla.py', outfn='test_0000_Bla.py')

In [10]:
fnlst = (filterfn(fn) for fn in rawlist)
fnlst = tuple(r for r in fnlst if not r is None)
fnlst

(datarecord_nt(tid='0000', comment='TestNotebook', fn='NBTest_0000_TestNotebook.py', outfn='test_0000_TestNotebook.py'),
 datarecord_nt(tid='032', comment='Analytics', fn='NBTest_032_Analytics.py', outfn='test_032_Analytics.py'),
 datarecord_nt(tid='033', comment='carbonpair', fn='NBTest_033_carbonpair.py', outfn='test_033_carbonpair.py'),
 datarecord_nt(tid='034', comment='Liquidity', fn='NBTest_034_Liquidity.py', outfn='test_034_Liquidity.py'),
 datarecord_nt(tid='035', comment='UsePositions', fn='NBTest_035_UsePositions.py', outfn='test_035_UsePositions.py'),
 datarecord_nt(tid='036', comment='LiquidityApprox', fn='NBTest_036_LiquidityApprox.py', outfn='test_036_LiquidityApprox.py'),
 datarecord_nt(tid='037', comment='Match', fn='NBTest_037_Match.py', outfn='test_037_Match.py'),
 datarecord_nt(tid='038', comment='CarbonPair', fn='NBTest_038_CarbonPair.py', outfn='test_038_CarbonPair.py'),
 datarecord_nt(tid='039', comment='Future', fn='NBTest_039_Future.py', outfn='test_039_Future.p

In [11]:
"jksjsdfngx[NOTEST]"[:-8]

'jksjsdfngx'

## Process files

In [12]:
def funcn(title):
    """convert a title into a function name"""
    title = title.strip()
    if title[-8:] == "[NOTEST]":
        prefix = "notest_"
        title = title[:-8].strip()
    else:
        prefix = "test_"
        
    funcn = title.lower()
    funcn = funcn.replace(" ", "_")
    funcn = prefix+funcn
    return funcn

assert funcn("Title") == "test_title"
assert funcn("Advanced Testing") == "test_advanced_testing"
#funcn("Asserting that the radius computes correctly")

In [13]:
def process_code(code, dr, srcpath=None, trgpath=None):
    """
    processes notebook code
    
    :code:    the code to be processed
    :dr:  the associated data record (datarecord_nt)
    :srcpath:   source path (info only)
    :trgpath:   target path (info only)
    """
    lines = code.splitlines()
    outlines = [
                 "# "+"-"*60,
                f"# Auto generated test file `{dr.outfn}`",
                 "# "+"-"*60,
                f"# source file   = {dr.fn}"
    ]
    if srcpath and srcpath != ".":
        outlines += [
                f"# source path   = {srcpath}"
        ]
    if trgpath and trgpath != ".":
        outlines += [
                f"# target path   = {srcpath}"
        ]
    outlines += [
        
                f"# test id       = {dr.tid}",
                f"# test comment  = {dr.comment}",
                 "# "+"-"*60,
                "","",
    ]
    is_precode = True
    for l in lines:
        if l[:4] == "# # ":
            print(f"""Processing "{l[4:]}" ({r.fn})""")
            outlines += [""]
        elif l[:5] == "# ## ":
            title = l[5:].strip()
            fcn = funcn(title)
            print(f"  creating function `{fcn}()` from section {title}")
            outlines += [
                 "",
                 "# "+"-"*60,
                f"# Test      {r.tid}",
                f"# File      {r.outfn}",
                f"# Segment   {title}",
                 "# "+"-"*60,
                f"def {fcn}():",
                 "# "+"-"*60,
            ]
            is_precode = False
        else:
            if is_precode:
                if l[:2] != "# ":
                    outlines += [l]
            else:
                outlines += ["    "+l]
    outcode = "\n".join(outlines)
    return outcode

In [14]:
for r in fnlst:
    code = fload(r.fn, SRCPATH, quiet=True)
    testcode = process_code(code, r, SRCPATH, TRGPATH)
    fsave(testcode, r.outfn, TRGPATH, quiet=True)
    #print(testcode)
    print(f"  saving generated test to {r.outfn}")

Processing "NBTest Test Notebook" (NBTest_0000_TestNotebook.py)
  creating function `test_segment_1()` from section Segment 1
  creating function `test_segment_2()` from section Segment 2
  creating function `notest_segment_3()` from section Segment 3 [NOTEST]
  saving generated test to test_0000_TestNotebook.py
Processing "Carbon Simulation - Test 32" (NBTest_032_Analytics.py)
  creating function `test_basic_arithmetic_functions()` from section Basic arithmetic functions
  creating function `test_advanced_functions()` from section Advanced functions
  saving generated test to test_032_Analytics.py
Processing "Carbon Simulation - Test 33 - CarbonPair" (NBTest_033_carbonpair.py)
  creating function `test_carbonpair()` from section CarbonPair
  saving generated test to test_033_carbonpair.py
Processing "Carbon Simulation - Test 34 - Liquidity" (NBTest_034_Liquidity.py)
  creating function `test_liquidity()` from section Liquidity
  saving generated test to test_034_Liquidity.py
Processin